In [1]:
import warnings
from math import*
from sqlalchemy import create_engine
import numpy as np
import pandas as pd
import time
import pickle
import random
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import scipy.io as scio
from scipy.stats import norm
import statsmodels.api as sm

1. 試用雅克比迭代法解如下線性方程組。
 


In [2]:

A=np.array([[3,-1,1],[1,3,1],[1,2,7]])
b=np.array([[7],[-3],[11]])
x0=np.array([[0],[0],[0]])
eps=10**-3
M=1000
def Jacob(A,b,x0,eps,M):
    N=np.diag(np.diag(A))
    P=N-A
    invN=np.linalg.inv(N)#N-1
    invNb=np.dot(invN,b)#b
    invNP=np.dot(invN,P)#M
    x=invNb+np.dot(invNP,x0)
    n=1
    while np.max(np.abs(x-x0))>=eps:
        x0=x
        x=invNb+np.dot(invNP,x0)
        n=n+1
        print(x)
        if n>=M:
            print("迭代次数太多")
            break
    root=x
    return root
root=Jacob(A,b,x0,eps,M)
print(root)

[[ 1.47619048]
 [-2.3015873 ]
 [ 1.52380952]]
[[ 1.05820106]
 [-2.        ]
 [ 2.01814059]]
[[ 0.99395314]
 [-2.02544722]
 [ 1.99168556]]
[[ 0.99428907]
 [-1.9952129 ]
 [ 2.00813447]]
[[ 0.99888421]
 [-2.00080785]
 [ 1.9994481 ]]
[[ 0.99991468]
 [-1.9994441 ]
 [ 2.00039021]]
[[ 1.00005523]
 [-2.00010163]
 [ 1.99985336]]
[[ 1.00005523]
 [-2.00010163]
 [ 1.99985336]]


2. 試用高斯—賽德爾迭代法求解第1題中的線性方程組。

In [3]:
def Guessseidei(A,b,x0,eps,M):
    N=np.tril(A)#A下三角
    P=N-A#计算P矩阵
    invN=np.linalg.inv(N)
    invNb=np.dot(invN,b)
    invNP=np.dot(invN,P)
    x=invNb+np.dot(invNP,x0)
    n=1#迭代此时
    while np.max(np.abs(x-x0))>=eps:
        x0=x
        x=invNb+np.dot(invNP,x0)
        n=n+1
        print(x)
        if n>=M:
            print('迭代次数太多')
            break
    root=x
    return x
root=Guessseidei(A,b,x0,eps,M)
print(root)

[[ 1.15873016]
 [-1.96825397]
 [ 1.96825397]]
[[ 1.02116402]
 [-1.99647266]
 [ 1.99596876]]
[[ 1.00251953]
 [-1.99949609]
 [ 1.99949609]]
[[ 1.00033594]
 [-1.99994401]
 [ 1.99993601]]
[[ 1.00003999]
 [-1.999992  ]
 [ 1.999992  ]]
[[ 1.00003999]
 [-1.999992  ]
 [ 1.999992  ]]


3. 請從銳思資料庫下載上證能源、上證資料、上證醫藥指數月數據，同時下載月三因子數據，建構中國股市的三因子模型，利用兩種迭代法估計三因子模型的參數。

In [4]:
data=pd.read_excel("股票三因子9.xls")
X1=data['市场指数'] #自变量1市场指数
X2 = data['上证材料指数月收益率_IdxMonRet'] #自变量2组合1
X3 = data['上证能源月收益率'] #自变量3组合2
X4 = data['上证医药月收益率'] #自变量4组合3
X5 =data['无风险利率']#自变量6无风险利率
X6 = data['市值因子__流通市值加权_Smb_tmv']#自变量7SMB
X7 = data['账面市值比因子__流通市值加权_Hml_tmv']#自变量8HML
Y = data['上证材料指数月收益率_IdxMonRet']-data['无风险利率'] #因变量ri-rf
Y1 = data['上证能源月收益率']-data['无风险利率'] #因变量ri-rf
Y2 = data['上证医药月收益率']-data['无风险利率']#因变量ri-rf
K=X1-X5
data['rm-rf']=K
y=np.sum(Y)
x1y=np.sum(K*Y)
x2y=np.sum(X6*Y)
x3y=np.sum(X7*Y)
n=len(X1)
x1=np.sum(K)
x2=np.sum(X6)
x3=np.sum(X7)
X11=np.sum(K**2)
X12=np.sum(K*X6)
X13=np.sum(K*X7)
X22=np.sum(X6**2)
X23=np.sum(X6*X7)
X33=np.sum(X7**2)
A=np.array([[n],[x1],[x2],[x3]])
B=np.array([[x1],[X11],[X12],[X13]])
C=np.array([[x2], [X12], [X22], [X23]])
D=np.array([[x3], [X13], [X23], [X33]])
H=np.array([[y], [x1y], [x2y], [x3y]])
E=np.hstack((A,B))
F =np.hstack((E, C))
G=np.hstack((F, D))
x0=np.array([[0],[0],[0],[0]])
eps=10**-3
M=1000
def Guessseidei(H,G,x0,eps,M):
    N=np.tril(G)#A下三角
    P=N-G#计算P矩阵
    invN=np.linalg.inv(N)
    invNH=np.dot(invN,H)
    invNP=np.dot(invN,P)
    x=invNH+np.dot(invNP,x0)
    n=1#迭代此时
    while np.max(np.abs(x-x0))>=eps:
        x0=x
        x=invNH+np.dot(invNP,x0)
        n=n+1
        if n>=M:
            print('迭代次数太多')
            break
    root=x
    return x
root=Guessseidei(H,G,x0,eps,M)
print(root)


def Jacob(G,H,x0,eps,M):
    N=np.diag(np.diag(G))
    P=N-G
    invN=np.linalg.inv(N)#N-1
    invNb=np.dot(invN,H)#b
    invNP=np.dot(invN,P)#M
    x=invNb+np.dot(invNP,x0)
    n=1
    while np.max(np.abs(x-x0))>=eps:
        x0=x
        x=invNb+np.dot(invNP,x0)
        n=n+1
        if n>=M:
            print("迭代次数太多")
            break
    root=x
    return root
root=Jacob(G,H,x0,eps,M)
print(root)

[[ 0.00125222]
 [ 1.18183108]
 [-0.02903433]
 [-0.16979507]]
[[ 0.00125335]
 [ 1.18181537]
 [-0.02884673]
 [-0.16968545]]
